In [1]:
from dask.distributed import Client, progress
from dask import delayed
import dask
#from larch.xafs import preedge
import larch.xafs as lx
import h5py, os, time, timeit, numpy as np
from numba import njit, prange
import matplotlib.pyplot as plt

[numexpr.utils] INFO : Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[numexpr.utils] INFO : NumExpr defaulting to 8 threads.
[dxchange.reader] WARNING : astropy module not found


In [ ]:
client = Client(threads_per_worker=4, n_workers=os.cpu_count()-1)
client

In [2]:
fn = '/media/xiao_usb/3D_trial_reg_scan_id_29773-29873_2020-06-01-20-38-09.h5'
f = h5py.File(fn, 'r')
# data = f['/registration_results/reg_results/registered_xanes3D'][:, 0, 50:200, 420:550]
data = f['/registration_results/reg_results/registered_xanes3D'][:, 0, :, :]
eng = f['/registration_results/reg_results/eng_list'][:]*1e3
f.close()
eng -= eng[0]

dim = data.shape
print(dim)

#ids = list(np.arange(20)) + list(np.arange(40, 49))
#x = eng[ids]
#y = data[ids, :].reshape([len(ids), -1])
y = data.reshape([dim[0], -1])

(101, 383, 1040)


In [ ]:
# single thread python
for ii in np.int32(np.arange(dim[1]*dim[2])):
    preedge(eng, mu=y[:, ii], e0=eng[40], step=None, nnorm=1,
             nvict=1, pre1=-150, pre2=-30, norm1=100, norm2=300)

In [4]:
dir(lx)

['ETOK',
 'FeffDatFile',
 'FeffPathGroup',
 'FeffRunner',
 'FeffitDataSet',
 'KTOE',
 'TransformGroup',
 '__DOC__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_larch_builtins',
 '_larch_groups',
 '_larch_init',
 '_sigma2_funcs',
 'autobk',
 'cauchy_wavelet',
 'constants',
 'deconvolve',
 'diffKKGroup',
 'diffkk',
 'estimate_noise',
 'etok',
 'feff6l',
 'feff8_xafs',
 'feff8l',
 'feff8lpath',
 'feffdat',
 'feffit',
 'feffit_dataset',
 'feffit_report',
 'feffit_transform',
 'feffpath',
 'feffrunner',
 'ff2chi',
 'find_e0',
 'find_exe',
 'fluo',
 'fluo_corr',
 'ftwindow',
 'guess_energy_units',
 'ktoe',
 'mback',
 'mback_norm',
 'path2chi',
 'pre_edge',
 'pre_edge_baseline',
 'preedge',
 'prepeaks_setup',
 'rebin_xafs',
 'set_xafsGroup',
 'sigma2_correldebye',
 'sigma2_debye',
 'sigma2_eins',
 'sigma2_models',
 'sort_xafs',
 'xafsft',
 'xafsutils',
 'xas_convolve',
 'xas_deconvolve',
 'xftf',
 'xftf_fast

In [6]:
# numba
local_preedge = lx.preedge
@njit
def pre_edge(eng, mu=None, e0=None, step=None, nnorm=None,
             nvict=None, pre1=None, pre2=None, norm1=None, norm2=None):
    local_preedge(eng, mu=mu, e0=e0, step=step, nnorm=nnorm,
             nvict=nvict, pre1=pre1, pre2=pre2, norm1=norm1, norm2=norm2)
for ii in prange(np.int32(dim[1]*dim[2])):
    pre_edge(eng, mu=y[:, ii], e0=eng[40], step=None, nnorm=1,
             nvict=1, pre1=-150, pre2=-30, norm1=100, norm2=300)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Untyped global name 'local_preedge': cannot determine Numba type of <class 'function'>

File "<ipython-input-6-226a46fa5b32>", line 6:
def pre_edge(eng, mu=None, e0=None, step=None, nnorm=None,
    <source elided>
             nvict=None, pre1=None, pre2=None, norm1=None, norm2=None):
    local_preedge(eng, mu=mu, e0=e0, step=step, nnorm=nnorm,
    ^


In [ ]:
results = []
print(time.asctime())
for ii in np.int32(np.arange(dim[1]*dim[2])):
    rlt = delayed(preedge)(eng, mu=y[:, ii], e0=eng[40], step=None, nnorm=1,
                           nvict=1, pre1=-150, pre2=-30, norm1=100, norm2=300)
    results.append(rlt)
%time dask.compute(*results, scheduler="processes")


print(time.asctime())

In [ ]:
client.close()

In [ ]:
import time
import random

def inc(x):
    time.sleep(random.random())
    return x + 1

def dec(x):
    time.sleep(random.random())
    return x - 1

def add(x, y):
    time.sleep(random.random())
    return x + y

In [ ]:
%%time
x = inc(1)
y = dec(2)
z = add(x, y)
z

In [ ]:
help(Client)

In [67]:
from larch.xafs import preedge
import h5py, os, time, timeit, numpy as np
import multiprocessing as mp
import matplotlib.pyplot as plt

fn = '/media/xiao_usb/3D_trial_reg_scan_id_29773-29873_2020-06-01-20-38-09.h5'
f = h5py.File(fn, 'r')
# data = f['/registration_results/reg_results/registered_xanes3D'][:, 0, 50:200, 420:550]
data = f['/registration_results/reg_results/registered_xanes3D'][:, 0, :, :]*1e4
eng = f['/registration_results/reg_results/eng_list'][:]*1e3
f.close()
eng -= eng[0]

dim = data.shape
print(dim)

ids = list(np.arange(20)) + list(np.arange(40, 49))
#x = eng[ids]
#y = data[ids, :].reshape([len(ids), -1])
x = eng
y = data.reshape([dim[0], -1])

# def preedge(x, amplitude, center, sigma, slope, intercept):
#     """wrapper to larch.pre_edge"""
#     pass

# def mp_fit(x, y, par_dict, par_bounds):
#     try:
#         return curve_fit(pre_peak, x, y, (par_dict['amplitude'], par_dict['center'], par_dict['sigma'],
#                                           par_dict['slope'], par_dict['intercept']), 
#                          bounds=par_bounds)
#     except:
#         return (None, None)
    
# pre_edge(energy, mu=None, group=None, e0=None, step=None, nnorm=None,
#              nvict=0, pre1=None, pre2=None, norm1=None, norm2=None,
#              make_flat=True, _larch=None)

# lrtz = LorentzianModel(prefix='lrtz_')
# pars = lrtz.make_params(amplitude=1.5, center=eng[49])
# pars['lrtz_amplitude'].set(value=1.5, min=0., max = 2)
# pars['lrtz_center'].set(value=eng[47], min=eng[40], max = eng[52])
# lin = LinearModel(prefix='lin_')
# pars.update(lin.make_params())
# pars['lin_slope'].set(value=0.1, min=-1, max=1)
# pars['lin_intercept'].set(value=0.1)
# mod = lrtz + lin
# #mod.fit(data[ids, 0, 0], pars, x=x, method='leastsq', nan_policy='omit')

# if __name__ == '__main__':
n_cpu = os.cpu_count()
par_dict = {'amplitude':1e-3, 'center':x[-1], 'sigma':3.0,
            'slope':0.1, 'intercept':0.0}
par_bounds = ((1e-5, x[-8], 0.1, 0, 0), (1e-2, x[-1], 10, 1, 2))
print(x.shape, y.shape)
print(time.asctime())

############ native python loop
# result = []
# for ii in range(dim[1]*dim[2]):
#     result.append(mp_fit(x, y[:, ii], par_dict))
# rlt = mp_fit(x, y[:, 1000], par_dict, par_bounds)
# pre_peak(x, amplitude, center, sigma, slope, intercept)
# print(rlt)
    
############ multiprocessing
def local_preedge(energy, mu, e0, step, nnorm, nvict, pre1,
                  pre2, norm1, norm2):
    return preedge(energy, mu, e0=e0, step=step, nnorm=nnorm,
                   nvict=nvict, pre1=pre1, pre2=pre2, norm1=norm1, norm2=norm2)
with mp.Pool(n_cpu-1) as pool:
    rlt = pool.starmap(local_preedge, [(x, y[:, ii], eng[43], None, 1,
                       0, -150, -50, 70, 430) for ii in np.int32(np.arange(dim[1]*dim[2]))])
pool.join()
pool.close()

print(time.asctime())
print(len(rlt))

# print(len(rlt[1000][0]))
# print(len(rlt[1000][1]))
# print((rlt[1000][0]))
# print((rlt[1000][1]))
# print((rlt[1000][0][0]))
# print(np.array(rlt).shape)

idx = int(108*dim[2] + 502) #100
print(rlt[idx]['norm_coefs'], rlt[idx]['precoefs'])
# plt.plot(x, prepeak(x, *rlt[idx][0]))
# plt.plot(x, y[:, idx])
# plt.show()

(101, 383, 1040)
(101,) (101, 398320)
Fri Jun 12 16:12:47 2020
Fri Jun 12 16:13:16 2020
398320
[7.6400961623798365, 0.030627643437632357] [-0.0415458   5.85828926]


In [27]:
idx = int(108*dim[2] + 502) 
plt.figure(1)
plt.plot(eng, rlt[idx]['norm_coefs'][0]+rlt[idx]['norm_coefs'][1]*eng, 'r')
plt.plot(eng, rlt[idx]['precoefs'][1]+rlt[idx]['precoefs'][0]*eng, 'g')
plt.plot(eng, y[:, idx])
print(eng[43])

132.99414


In [53]:
%matplotlib qt
plt.imshow(data[50]>10)

In [56]:
data_m = (data*(data[50]>10)).mean(axis=(1, 2))
rlt_m = preedge(x, data_m, e0=eng[43], step=None, nnorm=1,
                nvict=0, pre1=-150, pre2=-50, norm1=70, norm2=430)

In [62]:
plt.figure(2)
plt.plot(eng, (rlt_m['precoefs'][1]+rlt_m['precoefs'][0]*eng)+(rlt_m['norm_coefs'][0]+rlt_m['norm_coefs'][1]*eng), 'r')
plt.plot(eng, (rlt_m['precoefs'][1]+rlt_m['precoefs'][0]*eng), 'g')
plt.plot(eng, data_m)

In [47]:
print(rlt_m['norm_coefs'], rlt_m['precoefs'])

[1.2550789448639188, -0.0008303717102873441] [-2.61967429e-04  6.68449144e-01]


In [63]:
import numbers

In [66]:
help(numbers.Number)

Help on class Number in module numbers:

class Number(builtins.object)
 |  All numbers inherit from this class.
 |  
 |  If you just want to check if an argument x is a number, without
 |  caring what kind, use isinstance(x, Number).
 |  
 |  Data and other attributes defined here:
 |  
 |  __abstractmethods__ = frozenset()
 |  
 |  __hash__ = None



In [2]:
import numpy as np
print(np.arcsin(0.28))

0.28379410920832787


In [10]:
np.cos(np.pi/2+0.2838)

-0.280005655155147

In [13]:
0.56**2/4/np.pi*640

15.971516849157883

In [14]:
0.56**2

0.31360000000000005

In [3]:
a = {}
a[0] = 0
a[2] = 2
a[1] = 1
len(a.keys())

3

In [6]:
for ii in sorted(a.keys()):
    print(ii, a[ii])

0 0
1 1
2 2


In [10]:
import h5py, numpy as np
fn = '/media/xiao_usb/JuLi_2020Q1/2D_trial_reg_xanes_scan2_id_54308_2020-07-09-11-12-16.h5'
with h5py.File(fn, 'r') as f:
    """
    tem = f['/']
    for ii in tem:
        if 'processed_' in ii:
            print(ii)
            processed = ii
    tem = f[f'/{processed}/proc_spectrum']
    for ii in tem:
        print(ii)
    """
    print(np.min(f[f'/{processed}/proc_spectrum/whiteline_pos_fit']))

8350.0
